# 与其它量子化学输出文件解析器项目的基准测试

据我所知，有几个成熟的解析器将Python API用于量子化学输出文件，这些项目已经开发了几年。

1. [pymatgen](https://pymatgen.org/pymatgen.io.qchem.html)
2. [cclib](https://cclib.github.io/)
3. [openeye](https://docs.eyesopen.com/toolkits/python/oechemtk/molreadwrite.html) 

*注意*：项目[openeye](https://docs.eyesopen.com/toolkits/python/oechemtk/molreadwrite.html)根据其文档描述应该具有强大的功能，但不幸的是，我试图运行示例代码，并崩溃了内核，因此我无法在这里进行比较。

上述项目的主要功能与Molop的功能并不完全相同。该基准仅涵盖我们完成的解析器的性能和功能比较。目前，我们的项目仅支持对Gaussian和xTB相关的文件的解析，而其他项目对各种量子化学软件都有广泛的支持。我们将来可能会增加对其他软件格式的支持，但是目前，如果您需要解析其他软件输出，则可以使用上面的成熟项目来实现您的目标。

事实上，直到我决定开发 MolOP，我才意识到这些项目的存在。如果我早点知道它们，也许 MolOP 项目会是另一个工具，它只是提供了一种**从 xyz 坐标到分子图**的方法。当然，现在您也可以使用这个功能，我已经为它包装了一个单独的函数，您可以在 [**structure_recover_benchmark**](structure_recover_benchmark.md). 上了解它。

In [1]:
import time
from glob import glob

from cclib.io import ccopen
from pymatgen.io import gaussian

from molop import AutoParser
from molop.config import molopconfig


def timer_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        run_time = end_time - start_time
        print(f"Function '{func.__name__}' ran in {run_time:.6f} seconds")
        return result

    return wrapper


@timer_decorator
def pymatgen_parser(file_wildcard: str):
    result = []
    for file in glob(file_wildcard):
        try:
            result.append(gaussian.GaussianOutput(file))
        except Exception as e:
            print(f"Error parsing file {file}: {e}")
    return result


@timer_decorator
def cclib_parser(file_wildcard: str):
    result = []
    for file in glob(file_wildcard):
        try:
            result.append(ccopen(file).parse())
        except Exception as e:
            print(f"Error parsing file {file}: {e}")
    return result


@timer_decorator
def molop_parser(file_wildcard: str, quiet: bool = False, **kwargs):
    if quiet:
        molopconfig.quiet()
    else:
        molopconfig.verbose()
    return AutoParser(file_wildcard, **kwargs)

## 基准测试文件

基准测试文件在`tests/test_files`目录下。所有的文件都是合法的Gaussian输出文件。

In [2]:
print(AutoParser("../../tests/test_files/g16log/*.log").to_summary_df().to_markdown())

MolOP parsing with 16 jobs: 100%|██████████| 60/60 [00:01<00:00, 40.79it/s]
0 files failed to parse, 60 successfully parsed


|    | parser            | file_path                                                                                                                 | file_name                                                                    | file_format   | version                                    |   frame_index |   charge |   multiplicity | SMILES                                                                                                                                                                                                                                    | functional   | basis           | solvent_model   | solvent   |   temperature | status                                                                                                                                                                                                   |      ZPE |      TCE |      TCH |       TCG |      ZPE-Gas |        E-Gas |        H-Gas |        G-Gas |           sp |     HOMO |     LUMO |     G

## 时间基准测试

简而言之，MolOP 最多比 pymatgen 快 20 倍，比 cclib快 40 倍。

In [3]:
pymatgen_result = pymatgen_parser("../../tests/test_files/g16log/*.log")

Error parsing file ../../tests/test_files/g16log/anion_0167_opt_g16_nbo_sp.log: list index out of range


/home/tmj/miniconda3/envs/molop/lib/python3.8/site-packages/pymatgen/io/gaussian.py:1129: UserWarning: 
../../tests/test_files/g16log/cation_0407_opt_g16.log: Termination error or bad Gaussian output file !
  warnings.warn("\n" + self.filename + ": Termination error or bad Gaussian output file !")


Error parsing file ../../tests/test_files/g16log/radical_0554_opt_g16_nbo_sp.log: list index out of range
Function 'pymatgen_parser' ran in 2.810095 seconds


pymatgen 花了 2.1s 并产生了 2 个错误。

In [4]:
cclib_result = cclib_parser("../../tests/test_files/g16log/*.log")

[Gaussian ../../tests/test_files/g16log/dsgdb9nsd_000484-1+.log ERROR] Encountered error when parsing.
[Gaussian ../../tests/test_files/g16log/dsgdb9nsd_000484-1+.log ERROR] Last line read:  Rotational constants (GHZ):      ************     1.38922     1.38922



Error parsing file ../../tests/test_files/g16log/dsgdb9nsd_000484-1+.log: could not convert string to float: '************'
Function 'cclib_parser' ran in 5.245805 seconds


cclib 花了 4.1s 并产生了 1 个错误。

In [5]:
molop_result = molop_parser("../../tests/test_files/g16log/*.log", n_jobs=1)

MolOP parsing with single thread: 100%|██████████| 60/60 [00:02<00:00, 27.96it/s]
0 files failed to parse, 60 successfully parsed


Function 'molop_parser' ran in 2.148515 seconds


顺序 MolOP 大约花了2.1 s，没有产生任何错误，并可以通过自动并行化提高速度。

In [6]:
molop_result = molop_parser("../../tests/test_files/g16log/*.log")

MolOP parsing with 16 jobs: 100%|██████████| 60/60 [00:00<00:00, 147.37it/s]
0 files failed to parse, 60 successfully parsed


Function 'molop_parser' ran in 0.411318 seconds


Parallel MolOP took about 0.4s and raised no error. We can be faster if we want only the optimized structure.
并行化 MolOP 大约花了 0.4s，没有产生任何错误。如果只需要优化过的结构，还可以更快。

In [7]:
molop_result = molop_parser("../../tests/test_files/g16log/*.log", only_last_frame=True)

MolOP parsing with 16 jobs:   0%|          | 0/60 [00:00<?, ?it/s]

MolOP parsing with 16 jobs: 100%|██████████| 60/60 [00:00<00:00, 593.24it/s]
0 files failed to parse, 60 successfully parsed


Function 'molop_parser' ran in 0.105706 seconds


并行化 MolOP 处理最后一帧仅花了约0.1s，且没有产生错误。

## 信息基准测试

In [8]:
cclib_result[-1].getattributes().keys()

dict_keys(['atomcharges', 'atomcoords', 'atommasses', 'atomnos', 'charge', 'coreelectrons', 'dispersionenergies', 'enthalpy', 'entropy', 'freeenergy', 'geotargets', 'geovalues', 'grads', 'homos', 'metadata', 'moenergies', 'moments', 'mosyms', 'mult', 'natom', 'nbasis', 'nmo', 'optdone', 'optstatus', 'polarizabilities', 'pressure', 'rotconsts', 'scfenergies', 'scftargets', 'scfvalues', 'temperature', 'vibdisps', 'vibfconsts', 'vibfreqs', 'vibirs', 'vibrmasses', 'vibsyms', 'zpve'])

In [9]:
pymatgen_result[-1].as_dict().keys()

dict_keys(['has_gaussian_completed', 'nsites', 'unit_cell_formula', 'reduced_cell_formula', 'pretty_formula', 'is_pcm', 'errors', 'Mulliken_charges', 'elements', 'nelements', 'charge', 'spin_multiplicity', 'input', 'output', '@module', '@class'])

In [10]:
molop_result[-1][-1].to_dict().keys()

dict_keys(['smiles', 'atom_number', 'total_charge', 'total_multiplicity', 'atoms', 'coords', 'bonds', 'formal_charges', 'formal_spins', 'qm_software', 'qm_software_version', 'functional', 'basis', 'solvent_model', 'solvent', 'temperature', 'mulliken_charge', 'spin_densities', 'gradients', 'single_point_energy', 'zpve', 'energy_correction', 'enthalpy_correction', 'gibbs_free_energy_correction', 'U0', 'U_T', 'H_T', 'G_T', 'alpha_orbital_energies', 'beta_orbital_energies', 'alpha_homo', 'alpha_lumo', 'alpha_gap', 'beta_homo', 'beta_lumo', 'beta_gap', 'first_frequency', 'first_vibration_mode', 'second_frequency', 'second_vibration_mode', 'freqs', 'is_TS', 'spin_eginvalue', 'spin_multiplicity', 'dipole', 'quadrupole', 'octapole', 'hexadecapole', 'nbo_bond_order', 'wiberg_bond_order', 'mo_bond_order', 'atom_atom_overlap_bond_order', 'nbo_charges', 'lowdin_charges', 'hirshfeld_charges'])


pymatgen仅仅提取Gaussian输出文件中的基础信息。

cclib 对Gaussian输出文件进行了非常完整的解析，涵盖了其中的几乎所有内容，这很棒。不幸的是，~~某些字段的处理中有bug~~。

Molop对高斯输出文件的支持超过了pymatgen，但几乎不超过cclib的覆盖范围。

更重要的是，MolOP 提取的最小数据单元是文件中的每个合法帧，每个帧都以相同的规范被解析。其他项目将只处理整个文件。优化过程的结构对于建立分子几何数据集等任务同样重要。几何优化中的每一个框架都是一个合法的 SCF 收敛结构，并且提供了一个 DFT 级的分子力场，这对于神经网络势函数的拟合有很大的帮助。

In [11]:
molop_result = molop_parser("../../tests/test_files/g16log/*.log")
print(molop_result[9].to_summary_df().to_markdown(floatfmt=".6f"))

MolOP parsing with 16 jobs: 100%|██████████| 60/60 [00:00<00:00, 129.51it/s]
0 files failed to parse, 60 successfully parsed


Function 'molop_parser' ran in 0.467957 seconds
|    | parser            | file_path                                                                                             | file_name                                                | file_format   | version                                    |   frame_index |   charge |   multiplicity | SMILES                                               | functional   | basis   | solvent_model   | solvent   |   temperature | status                                                                                                                                                |      ZPE |      TCE |      TCH |      TCG |      ZPE-Gas |        E-Gas |        H-Gas |        G-Gas |           sp |      HOMO |      LUMO |      GAP |   first freq | first freq tag   |   second freq | second freq tag   | S**2   | S   |
|---:|:------------------|:------------------------------------------------------------------------------------------------------|:---------